In [1]:
#export 
from collections import defaultdict
from enum import IntFlag
from operator import itemgetter
import sys

import numpy as np
import pandas as pd

from loop.utils import merge_dicts, from_torch, autoformat, to_snake_case, classname, to_list


__all__ = ['RollingLoss', 'History', 'Average', 'StreamLogger', 'Group']

In [ ]:
#export
class Order(IntFlag):
    Unknown = -1
    Internal = 0
    Loss = 10
    Metric = 100
    Schedule = 200
    Tracker = 300
    Logging = 1000
    
    def __call__(self, index=0):
        return Order(self.value + index)
    
    @staticmethod
    def sort(items):
        ordered = [(getattr(item, 'order', Order.Unknown), item) for item in items]
        ordered.sort(key=itemgetter(0))
        return [item for _, item in ordered]

In [ ]:
#export
class Callback:
    """The base class inherited by callbacks.
    
    Provides a lot of hooks invoked on various stages of the training loop execution. The 
    signature of functions is as broad as possible to allow flexibility and customization 
    in descendant classes.
    """
    def training_started(self, **kwargs): pass
    def training_ended(self, **kwargs): pass
    def epoch_started(self, **kwargs): pass
    def epoch_ended(self, **kwargs): pass
    def phase_started(self, **kwargs): pass
    def phase_ended(self, **kwargs): pass
    def batch_started(self, **kwargs): pass
    def batch_ended(self, **kwargs): pass
    def before_forward(self, **kwargs): pass
    def after_forward(self, **kwargs): pass
    def before_backward(self, **kwargs): pass
    def after_backward(self, **kwargs): pass
    def interrupted(self, **kwargs): pass

In [ ]:
#export
class RollingLoss(Callback):
    """A callback that tracks model's loss.
    
    The loss is interpolated between current and next value to get a smoother loss curve.
    """
    order = Order.Loss()
    
    def __init__(self, smooth=0.98):
        self.smooth = smooth
    
    def batch_ended(self, phase, **kwargs):
        prev = phase.rolling_loss
        a = self.smooth
        avg_loss = a*prev + (1 - a)*phase.batch_loss
        debias_loss = avg_loss / (1 - a**phase.batch_index)
        phase.rolling_loss = avg_loss
        phase.update(debias_loss)
        
    def epoch_ended(self, phases, **kwargs):
        for phase in phases:
            phase.update_metric('loss', phase.last_loss)

In [ ]:
#export
class History(Callback):
    """A callback that collects model's metrics during its training."""
    
    order = Order.Tracker()
    
    def training_started(self, **kwargs):
        self.recorded = None
        self.epochs = []
    
    def epoch_ended(self, epoch, **kwargs):
        self.epochs.append(epoch)
        
    def training_ended(self, phases, **kwargs):
        epochs = {'epoch': np.array(self.epochs).astype(int)}
        metrics = [epochs] + [p.metrics_history for p in phases]
        data = pd.DataFrame(merge_dicts(metrics))
        data.reset_index(inplace=True, drop=True)
        self.recorded = data
        
    def plot(self, x='epoch', ys=('train_loss', 'valid_loss'), ax=None):
        return self.recorded.plot(x='epoch', y=to_list(ys), ax=ax)

In [ ]:
#export
class Average(Callback):
    """A callback that computes running average of metric.
    
    Params:
        metric_fn: Function accepting predictions and target, and returning a scalar value.
        name: Alias overriding metric name. If None, then a function name is used.
    
    """
    order = Order.Metric()
    
    def __init__(self, metric_fn: 'callable', alias: str=None):
        self.metric_fn = metric_fn
        self.name = alias or self.metric_fn.__name__
    
    def epoch_started(self, **kwargs):
        self.values = defaultdict(int)
        self.counts = defaultdict(int)
        
    def batch_ended(self, phase, output, target, **kwargs):
        metric = from_torch(self.metric_fn(output, target))
        self.counts[phase.name] += target.size(0)
        self.values[phase.name] += target.size(0) * metric
        
    def epoch_ended(self, phases, **kwargs):
        for phase in phases:
            metric = self.values[phase.name] / self.counts[phase.name]
            phase.update_metric(self.name, metric)

In [ ]:
#export
class StreamLogger(Callback):
    """Writes performance metrics collected during the training process into 
    list of streams.
    
    Parameters:
        streams: A list of file-like objects with `write()` method.
    
    """
    order = Order.Logging()
    
    def __init__(self, streams: list=None, log_every: int=1):
        self.streams = streams or [sys.stdout]
        self.log_every = log_every
        
    def epoch_ended(self, phases, epoch, **kwargs):
        metrics = merge_dicts([p.last_metrics for p in phases])
        values = [f'{k}={autoformat(v)}' for k, v in metrics.items()]
        values_string = ', '.join(values)
        self.write(f'Epoch: {epoch:4d} | {values_string}\n')
        
    def interrupted(self, exc, **kwargs):
        self.write(exc)
    
    def write(self, msg):
        for stream in self.streams:
            stream.write(msg)
            stream.flush()

In [ ]:
#export
class Group(Callback):
    """Wraps a list of callbacks and delegates calls to them.
    
    Each observer has a backward reference to its group via 'group' attribute. The group
    keeps a reference to the model which can be used by the 
    """
    def __init__(self, cbs, model=None):
        self._init(cbs)
        self.model = model
        
    def _init(self, cbs):
        if not cbs:
            self.callbacks = []
            self.named_callbacks = {}
        else:
            cbs = Order.sort(cbs)
            for cb in cbs:
                cb.group = self
            self.callbacks = cbs
            self.named_callbacks = {to_snake_case(classname(cb)): cb for cb in cbs}
        
    def add(self, cb, *cbs):
        cbs = [cb] + list(cbs)
        self._init(cbs)

    def training_started(self, **kwargs): self('training_started', **kwargs)    
    def training_ended(self, **kwargs): self('training_ended', **kwargs)
    def epoch_started(self, **kwargs): self('epoch_started', **kwargs)
    def epoch_ended(self, **kwargs): self('epoch_ended', **kwargs)
    def phase_started(self, **kwargs): self('phase_started', **kwargs)
    def phase_ended(self, **kwargs): self('phase_ended', **kwargs)
    def batch_started(self, **kwargs): self('batch_started', **kwargs)
    def batch_ended(self, **kwargs): self('batch_ended', **kwargs)
    def before_forward(self, **kwargs): self('before_forward', **kwargs)
    def after_forward(self, **kwargs): self('after_forward', **kwargs)
    def before_backward(self, **kwargs): self('before_forward', **kwargs)
    def after_backward(self, **kwargs): self('after_backward', **kwargs)
    def interrupted(self, **kwargs): self('interrupted', **kwargs)
        
    def __getattr__(self, item):
        if item in vars(self):
            return self.__dict__[item]
        if self._model is not None:
            return getattr(self._model, item)
        raise AttributeError(item)
    
    def __getitem__(self, item):
        item = to_snake_case(item)
        if item in self.named_callbacks:
            return self.named_callbacks[item]
        raise KeyError(
            f'callback name is not found: {item}; '
            f'available callbacks are: {list(sorted(self.named_callbacks))}')
    
    def __call__(self, name, **kwargs):
        for cb in self.callbacks:
            method = getattr(cb, name, None)
            if method is None:
                continue
            method(**kwargs)